In [1]:
import os
import glob
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import time
from random import choice
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [2]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
way = os.getcwd()

In [3]:
# headers и coockis для получения артикула и ссылки

headers_page = {
    'authority': 'catalog.sokolov.ru',
    'accept': 'application/json',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    'origin': 'https://sokolov.ru',
    'referer': 'https://sokolov.ru/jewelry-catalog/gold/?page=2',
    'sec-ch-ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'x-city-uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'x-country-uuid': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'x-region-uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'x-source': 'site',
}

params = {
    'page': '1',
    'per_page': '20',
    
}

catalog_gold = requests.get('https://catalog.sokolov.ru/api/v2/catalog/jewelry-catalog/gold/', params=params)

In [4]:
# headers для получаения инфы с ссылки

headers_url = {
    'authority': 'catalog.sokolov.ru',
    'accept': 'application/json',
    'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'content-type': 'application/json',
    'origin': 'https://sokolov.ru',
    'referer': 'https://sokolov.ru/jewelry-catalog/product/1012285-3/',
    'sec-ch-ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'x-city-uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'x-country-uuid': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'x-region-uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'x-source': 'site',
}

In [5]:
# headers и cookies для получения скидки

cookies_discont = {
    'guid_country': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'region_timezone': 'UTC%2B3%3A00',
    'fuser_id': 'a42ff5ab9cc1a7fd6a7c9c8887b92d3fa2433c387878b0fc6860564873ad3e29a%3A2%3A%7Bi%3A0%3Bs%3A8%3A%22fuser_id%22%3Bi%3A1%3Bs%3A32%3A%22de7a2b864fe0108aadf1b9885b8ffe60%22%3B%7D',
    'ab-test-user-id': '519e2a35485cbf40dc30d8f644590cbf995bd4fd165b788201b48387da3044a4a%3A2%3A%7Bi%3A0%3Bs%3A15%3A%22ab-test-user-id%22%3Bi%3A1%3Bs%3A32%3A%22ad3b5516f0260cf2ab50e5c73e889e62%22%3B%7D',
    '_userGUID': '0:llbx4b7o:qlwv_gb8EPSRUGJBVizaYWgacQYCXBlH',
    '_gcl_au': '1.1.558677429.1692080633',
    '_ym_uid': '1692080633101306662',
    '_ym_d': '1692080633',
    'tmr_lvid': 'e9600fe93abcdf9d9aaafa0c967ccdf1',
    'tmr_lvidTS': '1692080633202',
    'flocktory-uuid': '60efdf36-5dab-4787-a766-ce448f150b0e-6',
    'syte_uuid': '4ecfe650-3b34-11ee-86d7-6b708077495e',
    'adrcid': 'AtZv_rmyMET3A3qcFaqV7TA',
    'popmechanic_sbjs_migrations': 'popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1',
    'uxs_uid': '5032aaa0-3b34-11ee-8e27-1f685a3d14b9',
    'syte_ab_tests': '{}',
    'sklvt': '$2y$10$9q3XIpvh5shqDgeZrgYlp.mLWZ/Ol9wkNKYKcurQVxDPKNu/J.LEW',
    '_identity': 'f5a810731033be7dc7bcdd45a5d91a82335e4d8f222809f4b5693c7fc42dbb6fa%3A2%3A%7Bi%3A0%3Bs%3A9%3A%22_identity%22%3Bi%3A1%3Bs%3A82%3A%22%5B13706576%2C%22%242y%2410%249q3XIpvh5shqDgeZrgYlp.mLWZ%2FOl9wkNKYKcurQVxDPKNu%2FJ.LEW%22%2C31536000%5D%22%3B%7D',
    'guid_city': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'name_city': '%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3',
    'guid_region': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'inova_p_sid': '7z2jqbx_230914125535',
    '_gid': 'GA1.2.528491887.1696240461',
    'theme-mode': 'light',
    'DIGI_CARTID': '52756369189',
    'digi_uc': 'W1sidiIsIjg2OTI2MjI0MyIsMTY5NjMxODA0NDI5N10sWyJ2IiwiMTY5NDcwNDAxMiIsMTY5NjMxNjU0NDI1Nl0sWyJ2IiwiMjIyMTYxOTUxMSIsMTY5NjMxNjQ0NTAyN10sWyJ2IiwiNDExNDM0OTY2OCIsMTY5NjMxNjI4OTQ2MF0sWyJ2IiwiMzIzMjMzMzcwIiwxNjk1OTc4MjI4ODU4XSxbInYiLCIyMzU0OTUxMTI5IiwxNjk1OTc4MTkzNDI1XSxbInYiLCIxOTQyNDk0NTYzIiwxNjk1MjAxNDg4NTEwXSxbInYiLCIzMjIyNDE0MzM1IiwxNjk1MTk2ODM5ODgxXSxbInYiLCIxNzk1NzM0NjIiLDE2OTUxOTU0NjY5NzZdXQ==',
    'mindboxDeviceUUID': '5bd41fc7-bcf0-4931-a774-4bb47744fa76',
    'directCrm-session': '%7B%22deviceGuid%22%3A%225bd41fc7-bcf0-4931-a774-4bb47744fa76%22%7D',
    'iso_cookie': 'undefined',
    'UX_utm_source': 'undefined',
    'UX_utm_medium': 'undefined',
    '_ym_isad': '2',
    '_dvs': '0:lnbh1ttg:9zyVbH8yOgoaIA6UAObbuoZ~0xM0gj~k',
    'stimgs': '{%22sessionId%22:8891504%2C%22didReportCameraImpression%22:false%2C%22newUser%22:false}',
    '_ym_visorc': 'b',
    '_csrf': '63a971140d14ee88d2a353264cdd4c8f5fc33e69692ff5ff65df6791882a6357a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22xLpXkEgZ5-0H0eaCihFYUwDpxoG_pcVS%22%3B%7D',
}

headers_discont = {
    'Accept': 'application/json',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    # 'Cookie': 'guid_country=8aa15da9-92a4-4530-ab74-1992c973c539; region_timezone=UTC%2B3%3A00; fuser_id=a42ff5ab9cc1a7fd6a7c9c8887b92d3fa2433c387878b0fc6860564873ad3e29a%3A2%3A%7Bi%3A0%3Bs%3A8%3A%22fuser_id%22%3Bi%3A1%3Bs%3A32%3A%22de7a2b864fe0108aadf1b9885b8ffe60%22%3B%7D; ab-test-user-id=519e2a35485cbf40dc30d8f644590cbf995bd4fd165b788201b48387da3044a4a%3A2%3A%7Bi%3A0%3Bs%3A15%3A%22ab-test-user-id%22%3Bi%3A1%3Bs%3A32%3A%22ad3b5516f0260cf2ab50e5c73e889e62%22%3B%7D; _userGUID=0:llbx4b7o:qlwv_gb8EPSRUGJBVizaYWgacQYCXBlH; _gcl_au=1.1.558677429.1692080633; _ym_uid=1692080633101306662; _ym_d=1692080633; tmr_lvid=e9600fe93abcdf9d9aaafa0c967ccdf1; tmr_lvidTS=1692080633202; flocktory-uuid=60efdf36-5dab-4787-a766-ce448f150b0e-6; syte_uuid=4ecfe650-3b34-11ee-86d7-6b708077495e; adrcid=AtZv_rmyMET3A3qcFaqV7TA; popmechanic_sbjs_migrations=popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1; uxs_uid=5032aaa0-3b34-11ee-8e27-1f685a3d14b9; syte_ab_tests={}; sklvt=$2y$10$9q3XIpvh5shqDgeZrgYlp.mLWZ/Ol9wkNKYKcurQVxDPKNu/J.LEW; _identity=f5a810731033be7dc7bcdd45a5d91a82335e4d8f222809f4b5693c7fc42dbb6fa%3A2%3A%7Bi%3A0%3Bs%3A9%3A%22_identity%22%3Bi%3A1%3Bs%3A82%3A%22%5B13706576%2C%22%242y%2410%249q3XIpvh5shqDgeZrgYlp.mLWZ%2FOl9wkNKYKcurQVxDPKNu%2FJ.LEW%22%2C31536000%5D%22%3B%7D; guid_city=c2deb16a-0330-4f05-821f-1d09c93331e6; name_city=%D0%A1%D0%B0%D0%BD%D0%BA%D1%82-%D0%9F%D0%B5%D1%82%D0%B5%D1%80%D0%B1%D1%83%D1%80%D0%B3; guid_region=c2deb16a-0330-4f05-821f-1d09c93331e6; inova_p_sid=7z2jqbx_230914125535; _gid=GA1.2.528491887.1696240461; theme-mode=light; DIGI_CARTID=52756369189; digi_uc=W1sidiIsIjg2OTI2MjI0MyIsMTY5NjMxODA0NDI5N10sWyJ2IiwiMTY5NDcwNDAxMiIsMTY5NjMxNjU0NDI1Nl0sWyJ2IiwiMjIyMTYxOTUxMSIsMTY5NjMxNjQ0NTAyN10sWyJ2IiwiNDExNDM0OTY2OCIsMTY5NjMxNjI4OTQ2MF0sWyJ2IiwiMzIzMjMzMzcwIiwxNjk1OTc4MjI4ODU4XSxbInYiLCIyMzU0OTUxMTI5IiwxNjk1OTc4MTkzNDI1XSxbInYiLCIxOTQyNDk0NTYzIiwxNjk1MjAxNDg4NTEwXSxbInYiLCIzMjIyNDE0MzM1IiwxNjk1MTk2ODM5ODgxXSxbInYiLCIxNzk1NzM0NjIiLDE2OTUxOTU0NjY5NzZdXQ==; mindboxDeviceUUID=5bd41fc7-bcf0-4931-a774-4bb47744fa76; directCrm-session=%7B%22deviceGuid%22%3A%225bd41fc7-bcf0-4931-a774-4bb47744fa76%22%7D; iso_cookie=undefined; UX_utm_source=undefined; UX_utm_medium=undefined; _ym_isad=2; _dvs=0:lnbh1ttg:9zyVbH8yOgoaIA6UAObbuoZ~0xM0gj~k; stimgs={%22sessionId%22:8891504%2C%22didReportCameraImpression%22:false%2C%22newUser%22:false}; _ym_visorc=b; _csrf=63a971140d14ee88d2a353264cdd4c8f5fc33e69692ff5ff65df6791882a6357a%3A2%3A%7Bi%3A0%3Bs%3A5%3A%22_csrf%22%3Bi%3A1%3Bs%3A32%3A%22xLpXkEgZ5-0H0eaCihFYUwDpxoG_pcVS%22%3B%7D',
    'Referer': 'https://sokolov.ru/jewelry-catalog/product/1030855/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
    'X-City-Uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'X-Country-Uuid': '8aa15da9-92a4-4530-ab74-1992c973c539',
    'X-KL-kes-Ajax-Request': 'Ajax_Request',
    'X-Region-Uuid': 'c2deb16a-0330-4f05-821f-1d09c93331e6',
    'X-Source': 'site',
    'sec-ch-ua': '"Google Chrome";v="117", "Not;A=Brand";v="8", "Chromium";v="117"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

In [6]:
def get_article(page_max):
    article_all = []
    for page in tqdm(range(1, (page_max + 1))):
        params = {'page': str(page),'per_page': '44'}
        catalog_gold = requests.get('https://sokolov.ru/jewelry-catalog/gold/585/noinserts/?stock=Y', params=params)
        for i in range(0, 72):
            try:
                article = catalog_gold.json()['data'][i]['article']
            except:
                article = np.nan
            try:
                url = 'https://sokolov.ru/jewelry-catalog/product/' + catalog_gold.json()['data'][i]['url_code']
            except:
                url = np.nan
            article_all.append({'article': article,
                                'url' : url
                            })
        time.sleep(1)
    return article_all

In [7]:
options_chrome = webdriver.ChromeOptions()
options_chrome.add_argument('headless')

In [8]:
def get_info(article):
    data = []
    for art in tqdm(article):
        params_discont = {'url_code': art}
        url_json = requests.get(f'https://catalog.sokolov.ru/api/v2/products/{art}', headers=headers_url, timeout=None, verify=False)
        discount_json = requests.get('https://sokolov.ru/api/v5/products/sale-label/', params=params_discont, cookies=cookies_discont, headers=headers_discont, timeout=None, verify=False)
        time.sleep(0.5)
        try:
            name = url_json.json()['name']
        except:
            name = np.nan
        try:
            group = url_json.json()['category']
        except:
            group = np.nan
        try:
            weight = url_json.json()['characteristic_short'][1]['value']
        except:
            weight = np.nan
        try:
            price = url_json.json()['price']
        except:
            price = np.nan
        try:
            price_old = url_json.json()['old_price']
        except:
            price_old = np.nan
        try:
            insert = url_json.json()['characteristic_short'][2]['value']
        except:
            insert = np.nan
        try:
            metal = url_json.json()['characteristic_short'][0]['value']
        except:
            metal = np.nan
        try:
            lock_type_or_weaving_type = url_json.json()['characteristic'][0]['values'][4]['value']
        except:
            lock_type_or_weaving_type = np.nan
        try:
            discount = discount_json.json()['data'][0]['sticker_description'] + ' ' + discount_json.json()['data'][0]['sticker_text']
        except:
            discount = np.nan
        data.append(
            {
                'data': date_now,
                'competitor': 'Sokolov',
                'name': name,
                'group': group,
                'weight': weight,
                'price': price,
                'price_old': price_old,
                'discount': discount,
                'insert': insert,
                'metal': metal,
                'lock_type_or_weaving_type': lock_type_or_weaving_type,
                'article': art
            }
        )
    return data

In [9]:
options_chrome = webdriver.ChromeOptions()
options_chrome.add_argument('headless')
browser = webdriver.Chrome(options=options_chrome, service=Service(r'C:\Users\Trenkin.Sergey\.cache\selenium\chromedriver\win64\124.0.6367.201\chromedriver.exe')) 
links = []
home = 'https://sokolov.ru'
for number in tqdm(range(1, 22)):
    url_pars = str('https://sokolov.ru/jewelry-catalog/gold/noinserts/?stock=Y&page=' + str(number))
    try:
        browser.get(url_pars)
    except:
        browser.quit()
    WebDriverWait(browser, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'ProductList_products-list__We3gK')))
    pause_time = 0.5
    last_heigt = browser.execute_script("return document.body.scrollHeight")
    while True:
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause_time)
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_heigt:
            break
        last_heigt = new_height
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    for link in soup.find('div', class_='ProductList_products-list__We3gK').find_all('a', href=True):
        links.append(home + link['href'])

100%|██████████| 21/21 [01:01<00:00,  2.95s/it]


In [10]:
links = pd.DataFrame(links)
links.columns = ['url']
links_pars = links
links_pars['article'] = links['url'].str.replace('https://sokolov.ru/jewelry-catalog/product/', '')
links_pars['article'] = links_pars['article'].str.replace('/', '').str.strip()
# chunks = np.array_split(links_pars, 10)

In [11]:
links.to_csv(f'{way}\parsed_data\\links.csv', index = False)

In [12]:
slices = 5
chunks = np.array_split(links_pars['article'], slices)

In [13]:
n = 0
for i in range(n, slices):
    n += 1
    print(f'начало парсинга {n} части ссылок')
    df = get_info(chunks[i])
    df = pd.DataFrame(df)
    df.to_csv(rf'{way}\parsed_data\\' + f'bk_sokolov_{date_now}_{n}.csv', index=False)
    time.sleep(5)

начало парсинга 1 части ссылок


100%|██████████| 303/303 [04:23<00:00,  1.15it/s]


начало парсинга 2 части ссылок


100%|██████████| 303/303 [04:24<00:00,  1.15it/s]


начало парсинга 3 части ссылок


100%|██████████| 302/302 [04:21<00:00,  1.15it/s]


начало парсинга 4 части ссылок


100%|██████████| 302/302 [04:28<00:00,  1.13it/s]


начало парсинга 5 части ссылок


100%|██████████| 302/302 [04:24<00:00,  1.14it/s]


In [14]:
way = f'{way}\parsed_data'
# Удаляем файл с ссылками
os.remove(rf"{way}\links.csv")
# Указываем папку, в которой лежат файлы для объединения
os.chdir(rf'{way}')
# Указываем тип файлов, которые будем объединять
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
# Вывод имен файлов для объединения (для проверки)
all_filenames

['bk_sokolov_30.05.2024_1.csv',
 'bk_sokolov_30.05.2024_2.csv',
 'bk_sokolov_30.05.2024_3.csv',
 'bk_sokolov_30.05.2024_4.csv',
 'bk_sokolov_30.05.2024_5.csv']

In [15]:
data = pd.DataFrame()
for name in all_filenames:
    df = pd.read_csv(name)
    data = pd.concat([data, df], ignore_index=True)

In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   data                       1512 non-null   object
 1   competitor                 1512 non-null   object
 2   name                       1512 non-null   object
 3   group                      1509 non-null   object
 4   weight                     1512 non-null   object
 5   price                      1512 non-null   int64 
 6   price_old                  1512 non-null   int64 
 7   discount                   1179 non-null   object
 8   insert                     1512 non-null   object
 9   metal                      1512 non-null   object
 10  lock_type_or_weaving_type  649 non-null    object
 11  article                    1512 non-null   object
dtypes: int64(2), object(10)
memory usage: 141.9+ KB


In [17]:
data.sample(5)

,data,competitor,name,group,weight,price,price_old,discount,insert,metal,lock_type_or_weaving_type,article
1281,30.05.2024,Sokolov,Подвеска из золота с гравировкой,pendants,3.97 г,51750,114990,Дополнительная скидка по акции «Готовимся к ле...,Без вставок,Красное золото 585 пробы,NaN,036124
145,30.05.2024,Sokolov,"Цепь из белого золота, плетение Сингапур, 585 ...",chains,1.65 г,27000,59990,NaN,Без вставок,Белое золото 585 пробы,Сингапур,2106020-3
533,30.05.2024,Sokolov,Серьги из комбинированного золота,earrings,2.53 г,28800,63990,Дополнительная скидка по акции «Готовимся к ле...,Без вставок,Комбинированное золото 585 пробы,NaN,029465
1496,30.05.2024,Sokolov,Серьги из желтого золота,earrings,5.3 г,67500,149990,Дополнительная скидка по акции «Готовимся к ле...,Без вставок,Жёлтое золото 585 пробы,Английский,026747-2
169,30.05.2024,Sokolov,Колье из золота,necklaces,1.55 г,28350,62990,Дополнительная скидка по акции «Готовимся к ле...,Без вставок,Красное золото 585 пробы,NaN,070398


In [18]:
data.to_csv(rf'{way}\result\bk_sokolov_{date_now}.csv', index=False)
data.to_csv(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\raw_data\\' + f'bk_sokolov_{date_now}.csv', index=False)

In [19]:
data['discount'].value_counts()

discount
Дополнительная скидка по акции «Готовимся к лету!» ЕЩЁ -25%    1138
Дополнительная скидка по акции «Готовимся к лету!» ЕЩЁ -10%      26
Дополнительная скидка по акции «Готовимся к лету!» ЕЩЁ -20%      15
Name: count, dtype: int64

Для удаления файлов внутри папки:

In [20]:
def delete_files_in_folder(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path):
                os.remove(file_path)
        except Exception as e:
            print(f'Ошибка при удалении файла {file_path}. {e}')

In [21]:
delete_files_in_folder(f'{way}')